# Interfacing with R via rpy2

In [4]:
!wget http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/20130502.phase3.sequence.index

--2023-07-01 03:18:50--  http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/20130502.phase3.sequence.index
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.140
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.140|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67057948 (64M)
Saving to: '20130502.phase3.sequence.index'

     0K .......... .......... .......... .......... ..........  0% 99.9K 10m55s
    50K .......... .......... .......... .......... ..........  0%  300K 7m16s
   100K .......... .......... .......... .......... ..........  0%  782K 5m19s
   150K .......... .......... .......... .......... ..........  0%  415K 4m38s
   200K .......... .......... .......... .......... ..........  0%  304K 4m25s
   250K .......... .......... .......... .......... ..........  0%  308K 4m16s
   300K .......... .......... .......... .......... ..........  0%  308K 4m10s
   350K .......... .......... .......... ..